# Spark Dataframe Basics

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [3]:
df = spark.read.json('people.json')

In [4]:
# Notice SPARK replaces missing data with the word 'null'
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [5]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [6]:
df.columns

['age', 'name']

In [9]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [10]:
df.describe().show() # Only shows numeric columns' values!

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [11]:
from pyspark.sql.types import (StructField,StringType,
                               IntegerType,StructType)

In [12]:
# Change age schema into an INT and can have nulls (True)
# Change name schema into STR and can have nulls
data_schema = [StructField('age',IntegerType(),True),
              StructField('name',StringType(),True)]

In [13]:
final_struc = StructType(fields=data_schema)

In [14]:
df = spark.read.json('people.json',schema=final_struc)

In [15]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [16]:
df['age'] # This is a column

Column<b'age'>

In [17]:
df.select('age') # This is a dataframe

DataFrame[age: int]

In [18]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [19]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [20]:
df.head(2)[0]

Row(age=None, name='Michael')

In [22]:
df.select(['age','name'])

DataFrame[age: int, name: string]

In [23]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [26]:
# Creating a new column based on an existing one
df.withColumn('doubleAge',df['age']*2).show()

# WIll not save!

+----+-------+---------+
| age|   name|doubleAge|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



In [28]:
# Rename a column
df.withColumnRenamed('age','newAge').show()

+------+-------+
|newAge|   name|
+------+-------+
|  null|Michael|
|    30|   Andy|
|    19| Justin|
+------+-------+



In [29]:
# Register dataframe as a SQL Temp View
df.createOrReplaceTempView('people')

In [30]:
results = spark.sql("SELECT * FROM people")

In [31]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [32]:
newResults = spark.sql("SELECT * FROM people WHERE age = 30")

In [33]:
newResults.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

